<a href="https://colab.research.google.com/github/kjain1810/aicrowd/blob/master/ORIENTME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#preprocess data before NN

##install

In [ ]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install scikit-learn 
!{sys.executable} -m pip install matplotlib tqdm
!{sys.executable} -m pip install opencv-python

##data import

In [38]:
# #Donwload the datasets
!rm -rf data/
!mkdir data/

!curl https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/orientme/v0.2/training.tar.gz -o data/training.tar.gz
!curl https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/orientme/v0.2/test.tar.gz -o data/test.tar.gz
!curl https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/orientme/v0.2/sample_submission.csv -o data/sample_submission.csv
!tar xvzf data/training.tar.gz -C data/
!tar xvzf data/test.tar.gz -C data/


Streaming output truncated to the last 5000 lines.
images/008254.jpg
images/007167.jpg
images/008532.jpg
images/007601.jpg
images/005016.jpg
images/005002.jpg
images/008526.jpg
images/007615.jpg
images/009638.jpg
images/008240.jpg
images/007173.jpg
images/005764.jpg
images/006245.jpg
images/009176.jpg
images/008268.jpg
images/005994.jpg
images/006523.jpg
images/009610.jpg
images/007629.jpg
images/006537.jpg
images/009604.jpg
images/005980.jpg
images/005758.jpg
images/006251.jpg
images/009162.jpg
images/008297.jpg
images/009189.jpg
images/009837.jpg
images/009823.jpg
images/008283.jpg
images/005957.jpg
images/007198.jpg
images/006286.jpg
images/007826.jpg
images/007832.jpg
images/006292.jpg
images/005943.jpg
images/005214.jpg
images/007403.jpg
images/008730.jpg
images/007365.jpg
images/008056.jpg
images/009348.jpg
images/005572.jpg
images/005566.jpg
images/007371.jpg
images/008042.jpg
images/006709.jpg
images/007417.jpg
images/008724.jpg
images/005200.jpg
images/009412.jpg
images/006721

In [39]:
TRAINING_IMAGES_FOLDER = "data/training/images/"
TRAINING_LABELS_PATH = "data/training/labels.csv"
TEST_IMAGES_FOLDER = "data/images"
SAMPLE_SUBMISSION_FILE_PATH = "data/sample_submission.csv"


##import


In [ ]:
import os
import tqdm

import numpy as np
import pandas as pd
import cv2

from PIL import Image

from google.colab.patches import cv2_imshow

import glob

from google.colab import files

##pre-process data


###get corners of image

In [40]:
def get_corner(image):
    morph = im.copy()

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
    morph = cv2.morphologyEx(morph, cv2.MORPH_CLOSE, kernel)
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

    gradient_image = cv2.morphologyEx(morph, cv2.MORPH_GRADIENT, kernel)

    image_channels = np.split(np.asarray(gradient_image), 3, axis=2)

    channel_height, channel_width, _ = image_channels[0].shape

    for i in range(0, 3):
        _, image_channels[i] = cv2.threshold(~image_channels[i], 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY)
        image_channels[i] = np.reshape(image_channels[i], newshape=(channel_height, channel_width, 1))

    image_channels = np.concatenate((image_channels[0], image_channels[1], image_channels[2]), axis=2)

    lower = np.array([0, 0, 0])
    upper = np.array([254, 254, 254])
    ret = cv2.inRange(image_channels, lower, upper)
    return ret

###counting colors

In [41]:
def count_red(image, debug=False):
    img_hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
    
    lower_red = np.array([0,80,100])
    upper_red = np.array([5,200,255])
    mask0 = cv2.inRange(img_hsv, lower_red, upper_red)
    lower_red = np.array([175,80,100])
    upper_red = np.array([180,200,255])
    mask1 = cv2.inRange(img_hsv, lower_red, upper_red)
    mask = mask1 + mask0

    maskedge = get_corner(image)

    mask = mask - maskedge
    mask = np.maximum(mask, np.zeros((512, 512)))
    if debug:
        print("red: ")
        cv2_imshow(mask)
    num_col = np.sum(mask > 0)
    return num_col

In [42]:
def count_white(image, debug=False):
    lower_white = np.array([240, 240, 240])
    upper_white = np.array([255, 255, 255])
    mask = cv2.inRange(im, lower_white, upper_white)
    num_col = np.sum(mask > 0)
    if debug:
        print("white: ")
        cv2_imshow(mask)
    return num_col

In [43]:
def count_blue(image, debug=False):
    img_hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
    lower_blue=np.array([110, 150, 80],np.uint8)
    upper_blue=np.array([120, 220, 255],np.uint8)
    mask = cv2.inRange(img_hsv, lower_blue, upper_blue)
    if debug:
        print("blue: ")
        cv2_imshow(mask)
    num_col = np.sum(mask > 0)
    return num_col

In [44]:
def count_orange(image, debug=False):
    img_hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
    
    lower_red = np.array([5,50,50])
    upper_red = np.array([15,255,255])
    mask0 = cv2.inRange(img_hsv, lower_red, upper_red)
    lower_red = np.array([165,50,50])
    upper_red = np.array([175,255,255])
    mask1 = cv2.inRange(img_hsv, lower_red, upper_red)
    mask = mask1 + mask0

    maskedge = get_corner(image)

    mask = mask - maskedge
    mask = np.maximum(mask, np.zeros((512, 512)))
    if debug:
        print("orange: ")
        cv2_imshow(mask)
    num_col = np.sum(mask > 0)
    return num_col

In [45]:
def count_green(image, debug=False):
    img_hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
    lower_yellow=np.array([50, 100, 40],np.uint8)
    upper_yellow=np.array([70, 255, 255],np.uint8)
    mask = cv2.inRange(img_hsv, lower_yellow, upper_yellow)
    if debug:
        print("green: ")
        cv2_imshow(mask)
    num_col = np.sum(mask > 0)
    return num_col

In [46]:
def count_yellow(image, debug=False):
    img_hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
    lower_yellow=np.array([20, 100, 100],np.uint8)
    upper_yellow=np.array([30, 255, 255],np.uint8)
    mask = cv2.inRange(img_hsv, lower_yellow, upper_yellow)
    if debug:
        print("yellow: ")
        cv2_imshow(mask)
    num_col = np.sum(mask > 0)
    return num_col

###main

In [47]:
def pre_process_data_X(image):
    cols_here = np.zeros((6))
    cols_here[0] = count_red(image)
    cols_here[1] = count_white(image)
    cols_here[2] = count_blue(image)
    cols_here[3] = count_yellow(image)
    cols_here[4] = count_green(image)
    cols_here[5] = count_orange(image)
    return cols_here

In [ ]:
training_labels_df = pd.read_csv(TRAINING_LABELS_PATH)

ALL_DATA = np.zeros((5000, 7))

counter = 0

for _idx, row in tqdm.tqdm(training_labels_df.iterrows(), total=training_labels_df.shape[0]):
    filepath = os.path.join(
        TRAINING_IMAGES_FOLDER,
        row.filename
    )
    im = cv2.imread(filepath)
    
    data_X = pre_process_data_X(im)
    
    for i in range(6):
        ALL_DATA[counter][i] = data_X[i]
    ALL_DATA[counter][6] = row.xRot/360
    
    counter += 1

print(ALL_DATA)

##Checking the data

In [ ]:
for i in range(20):
    print(ALL_DATA[i][6])

In [ ]:
df = pd.DataFrame(data=ALL_DATA, columns=["red", "white", "blue", "yellow", "green", "orange", "xRot"])

In [ ]:
df.head

###download for future

In [ ]:
df.to_csv("orientme_first_preprocessing.csv", index=False)

In [ ]:
files.download('orientme_first_preprocessing.csv')

##load preprocessed data

In [ ]:
data=pd.read_csv("orientme_first_preprocessing.csv")

In [ ]:
data.head()

##remove noises

In [57]:
def remove_noise(row):
    # print(row)
    indexes = ["red", "white", "blue", "yellow", "green", "orange"]
    for x in indexes:
        if row[x] <= 1000:
            row[x] = 0
    return row

In [ ]:
data.apply(lambda rw: remove_noise(rw), axis=1)

In [ ]:
data.head(10)

In [58]:
def remove_orange_noise(row):
    if row["red"] - row["orange"] >= 2000:
        row["orange"] = 0
    elif row["orange"] - row["red"] >= 1000:
        row["red"] = 0
    return row

In [ ]:
data.apply(lambda rw: remove_orange_noise(rw), axis=1)

In [ ]:
data.loc[data["orange"] >= 5000].loc[data["orange"] <= 9000]

In [59]:
def remove_red_when_orange(row):
    if row["orange"] >= 5000:
        row["red"] = 0
    return row

In [ ]:
data.apply(lambda rw: remove_red_when_orange(rw), axis=1)

##download this data now

In [ ]:
data.to_csv("orientme_second_preprocessing.csv", index=False)

In [ ]:
files.download('orientme_second_preprocessing.csv')

##preprocess test set

In [48]:
TEST_DATA = np.zeros((5001, 6))
TEST_FILENAMES = []

counter = 0

for _test_image_path in tqdm.tqdm(glob.glob(os.path.join(TEST_IMAGES_FOLDER, "*.jpg"))):
    filename = os.path.basename(_test_image_path)
    im = cv2.imread(_test_image_path)
    
    data_X = pre_process_data_X(im)
    for i in range(6):
        TEST_DATA[counter][i] = data_X[i]
    counter += 1
    # TEST_DATA.append(data_X)
    TEST_FILENAMES.append(filename)


100%|██████████| 5001/5001 [02:00<00:00, 41.67it/s]


In [49]:
TEST_DATA = TEST_DATA.astype(int)

In [50]:
print(TEST_DATA)

[[ 2510     0 23147 13010     1  3094]
 [10915   228 24357     0    11 10048]
 [28531   222     0     0 11853  5424]
 ...
 [30038     0     0   688  6882  4538]
 [ 4563 29739    68     0   515  4586]
 [ 7430     6 27653   211     6  6505]]


In [62]:
test_dataframe = pd.DataFrame(data=TEST_DATA, columns=["red", "white", "blue", "yellow", "green", "orange"])
test_dataframe["filename"] = TEST_FILENAMES

In [63]:
test_dataframe.head()

,red,white,blue,yellow,green,orange,filename
0,2510,0,23147,13010,1,3094,009805.jpg
1,10915,228,24357,0,11,10048,009738.jpg
2,28531,222,0,0,11853,5424,008471.jpg
3,9985,917,15056,0,17,8764,008284.jpg
4,24544,0,145,15079,96,4428,006827.jpg


In [66]:
test_dataframe.to_csv("orientme_testdata_first_preprocessing.csv", index=False)

In [67]:
files.download("orientme_testdata_first_preprocessing.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [68]:
df=pd.read_csv("/content/orientme_testdata_first_preprocessing.csv")

In [69]:
df.loc[df["orange"]<5000]

,red,white,blue,yellow,green,orange,filename
0,2510,0,23147,13010,1,3094,009805.jpg
4,24544,0,145,15079,96,4428,006827.jpg
6,26020,0,117,12185,259,4628,005555.jpg
7,4037,0,27451,5819,0,4295,006972.jpg
8,24832,0,127,14960,132,4625,007767.jpg
...,...,...,...,...,...,...,...
4993,3034,0,20969,15171,2,3610,008965.jpg
4995,17085,0,3240,19311,4,4386,005406.jpg
4997,25585,0,96,12566,204,4508,005433.jpg
4998,30038,0,0,688,6882,4538,006821.jpg


In [70]:
df.apply(lambda rw: remove_noise(rw), axis=1)

,red,white,blue,yellow,green,orange,filename
0,2510,0,23147,13010,0,3094,009805.jpg
1,10915,0,24357,0,0,10048,009738.jpg
2,28531,0,0,0,11853,5424,008471.jpg
3,9985,0,15056,0,0,8764,008284.jpg
4,24544,0,0,15079,0,4428,006827.jpg
...,...,...,...,...,...,...,...
4996,30399,0,0,4643,2885,5330,008725.jpg
4997,25585,0,0,12566,0,4508,005433.jpg
4998,30038,0,0,0,6882,4538,006821.jpg
4999,4563,29739,0,0,0,4586,009709.jpg


In [71]:
df.loc[df["red"] <= 1000]

,red,white,blue,yellow,green,orange,filename


In [72]:
df.apply(lambda rw: remove_orange_noise(rw), axis=1)

,red,white,blue,yellow,green,orange,filename
0,2510,0,23147,13010,1,3094,009805.jpg
1,10915,228,24357,0,11,10048,009738.jpg
2,28531,222,0,0,11853,0,008471.jpg
3,9985,917,15056,0,17,8764,008284.jpg
4,24544,0,145,15079,96,0,006827.jpg
...,...,...,...,...,...,...,...
4996,30399,0,1,4643,2885,0,008725.jpg
4997,25585,0,96,12566,204,0,005433.jpg
4998,30038,0,0,688,6882,0,006821.jpg
4999,4563,29739,68,0,515,4586,009709.jpg


In [73]:
df.apply(lambda rw: remove_red_when_orange(rw), axis=1)

,red,white,blue,yellow,green,orange,filename
0,2510,0,23147,13010,1,3094,009805.jpg
1,0,228,24357,0,11,10048,009738.jpg
2,0,222,0,0,11853,5424,008471.jpg
3,0,917,15056,0,17,8764,008284.jpg
4,24544,0,145,15079,96,4428,006827.jpg
...,...,...,...,...,...,...,...
4996,0,0,1,4643,2885,5330,008725.jpg
4997,25585,0,96,12566,204,4508,005433.jpg
4998,30038,0,0,688,6882,4538,006821.jpg
4999,4563,29739,68,0,515,4586,009709.jpg


###download after second preprocess

In [74]:
df.to_csv("orientme_testdata_second_preprocessing.csv")

In [75]:
files.download("orientme_testdata_second_preprocessing.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#NN

##imports

In [76]:
import os
import tqdm

import numpy as np
import pandas as pd
import cv2

import glob

from google.colab import files

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
import plotly.graph_objects as go

##load preprocessed data

In [77]:
train = pd.read_csv("orientme_second_preprocessing.csv");
test = pd.read_csv("orientme_testdata_second_preprocessing.csv")

In [78]:
train.head()

,red,white,blue,yellow,green,orange,xRot
0,6643.0,26875.0,0.0,0.0,3450.0,4837.0,0.935664
1,2910.0,0.0,21659.0,15035.0,0.0,3433.0,0.412457
2,30738.0,0.0,0.0,3119.0,4922.0,0.0,0.680060
3,27567.0,0.0,0.0,10954.0,0.0,0.0,0.618613
4,9341.0,0.0,12511.0,19476.0,0.0,0.0,0.478773


In [79]:
test.head()

,Unnamed: 0,red,white,blue,yellow,green,orange,filename
0,0,2510,0,23147,13010,1,3094,009805.jpg
1,1,10915,228,24357,0,11,10048,009738.jpg
2,2,28531,222,0,0,11853,5424,008471.jpg
3,3,9985,917,15056,0,17,8764,008284.jpg
4,4,24544,0,145,15079,96,4428,006827.jpg


##split traintest

In [80]:
training_set, validation_set= train_test_split(train, test_size=0.2, random_state=42)

In [81]:
X_train,y_train = training_set.iloc[:,:-1],training_set.iloc[:,-1]
X_val,y_val = validation_set.iloc[:,:-1],validation_set.iloc[:,-1]


In [82]:
X_val.shape

(1000, 6)

In [85]:
file_names = test["filename"]

In [87]:
file_names.head()

0    009805.jpg
1    009738.jpg
2    008471.jpg
3    008284.jpg
4    006827.jpg
Name: filename, dtype: object

In [92]:
test = test.drop(columns=["filename"])

In [93]:
test = test.loc[:, ~test.columns.str.contains('^Unnamed')]


In [94]:
test.head()

,red,white,blue,yellow,green,orange
0,2510,0,23147,13010,1,3094
1,10915,228,24357,0,11,10048
2,28531,222,0,0,11853,5424
3,9985,917,15056,0,17,8764
4,24544,0,145,15079,96,4428


##data normalization

In [95]:
train_col_max = train.max()
train_max = train_col_max.max()
test_col_max = test.max()
test_max = test_col_max.max()
total_max = max(test_max, train_max)

In [96]:
print(total_max)

30998.0


In [97]:
X_train = X_train / total_max
X_val = X_val / total_max
test = test / total_max

In [98]:
X_train.head()

,red,white,blue,yellow,green,orange
4227,0.000000,0.677302,0.355120,0.000000,0.000000,0.238144
4676,0.171463,0.000000,0.494129,0.600781,0.000000,0.122072
800,0.169172,0.902155,0.000000,0.000000,0.073682,0.149816
3671,0.885541,0.000000,0.000000,0.335118,0.000000,0.000000
4193,0.939835,0.000000,0.000000,0.000000,0.374444,0.000000


##train model

In [139]:
def create_model():
    model = Sequential()
    model.add(Dense(32, input_dim=6, activation='relu', name='dense_1'))
    model.add(Dense(32, activation='relu', name='dense_2'))
    model.add(Dense(16, activation='relu', name='dense_3'))
    model.add(Dense(8, activation='relu', name='dense_4'))
    model.add(Dense(1, activation='sigmoid', name='dense_output'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model
model = Sequential()
model = create_model()
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                224       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_output (Dense)         (None, 1)                 9         
Total params: 1,953
Trainable params: 1,953
Non-trainable params: 0
_________________________________________________________________


In [140]:
history = model.fit(X_train, y_train, epochs=400, validation_split=0.05)


Epoch 1/400
119/119 [==============================] - 0s 2ms/step - loss: 0.0528 - mae: 0.1696 - val_loss: 0.0385 - val_mae: 0.1330
Epoch 2/400
119/119 [==============================] - 0s 1ms/step - loss: 0.0289 - mae: 0.0988 - val_loss: 0.0278 - val_mae: 0.0888
Epoch 3/400
119/119 [==============================] - 0s 2ms/step - loss: 0.0245 - mae: 0.0766 - val_loss: 0.0303 - val_mae: 0.0803
Epoch 4/400
119/119 [==============================] - 0s 2ms/step - loss: 0.0235 - mae: 0.0700 - val_loss: 0.0242 - val_mae: 0.0672
Epoch 5/400
119/119 [==============================] - 0s 2ms/step - loss: 0.0225 - mae: 0.0630 - val_loss: 0.0297 - val_mae: 0.0733
Epoch 6/400
119/119 [==============================] - 0s 2ms/step - loss: 0.0221 - mae: 0.0598 - val_loss: 0.0256 - val_mae: 0.0632
Epoch 7/400
119/119 [==============================] - 0s 1ms/step - loss: 0.0222 - mae: 0.0624 - val_loss: 0.0232 - val_mae: 0.0644
Epoch 8/400
119/119 [==============================] - 0s 2ms/step - 

In [141]:
fig = go.Figure()
fig.add_trace(go.Scattergl(y=history.history['loss'],
                    name='Train'))
fig.add_trace(go.Scattergl(y=history.history['val_loss'],
                    name='Valid'))
fig.update_layout(height=500, width=700,
                  xaxis_title='Epoch',
                  yaxis_title='Loss')
fig.show()


In [142]:
mse_nn, mae_nn = model.evaluate(X_val, y_val)
print('Mean squared error on test data: ', mse_nn)

32/32 [==============================] - 0s 840us/step - loss: 0.0219 - mae: 0.0473
Mean squared error on test data:  0.021873170509934425


##predict

In [103]:
test.head()

,red,white,blue,yellow,green,orange
0,0.080973,0.000000,0.746726,0.419704,0.000032,0.099813
1,0.352119,0.007355,0.785760,0.000000,0.000355,0.324150
2,0.920414,0.007162,0.000000,0.000000,0.382380,0.174979
3,0.322118,0.029583,0.485709,0.000000,0.000548,0.282728
4,0.791793,0.000000,0.004678,0.486451,0.003097,0.142848


In [104]:
predictions = model.predict(test)

In [105]:
predictions = predictions * 360

In [106]:
predictions

array([[139.90222 ],
       [ 62.823677],
       [282.7281  ],
       ...,
       [237.95453 ],
       [278.92245 ],
       [ 83.54799 ]], dtype=float32)

In [108]:
answers = pd.DataFrame(predictions, columns=["xRot"])
answers["filename"] = file_names

In [109]:
answers.head()

,xRot,filename
0,139.902222,009805.jpg
1,62.823677,009738.jpg
2,282.728088,008471.jpg
3,45.544369,008284.jpg
4,202.220810,006827.jpg


In [110]:
answers.to_csv("submission.csv")

In [111]:
files.download("submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>